In [1]:
import barzur20aft
import compiler
import numpy as np
import pandas

In [2]:
def bitcoin_mdp(*args, **kwargs):
    model = barzur20aft.Bitcoin(*args, **kwargs)
    c = compiler.Compiler(model)
    return c.mdp()

In [3]:
alpha = [0.01, 0.05, 0.1, 0.2, 0.3, 0.35, 0.4, 0.45, 0.49]
ptmdp = []
mdp = []
for a in alpha:
    print(a)
    m = bitcoin_mdp(alpha=a, gamma=1, maximum_fork_length=25)
    mdp.append(m)
    ptmdp.append(barzur20aft.ptmdp(m, horizon=100))

0.01
0.05
0.1
0.2
0.3
0.35
0.4
0.45
0.49


In [4]:
vi = []
for i, m in enumerate(ptmdp):
    res = m.value_iteration(value_eps=0.01)
    vi.append(res)
    v = res["vi_value"]
    p = res["vi_policy"]
    iter = res["vi_iter"]
    print(
        f"alpha={alpha[i]:.2f} iter={iter:3d} v[0]={v[0]:.2f} p={p[:10]} {sum(p)} {hash(p.tobytes()) % 10000}"
    )

alpha=0.01 iter= 30 v[0]=0.14 p=[0 0 1 0 3 0 1 0 0 0] 1046 246
alpha=0.05 iter=195 v[0]=3.23 p=[0 0 1 0 3 0 1 0 0 0] 949 9647
alpha=0.10 iter=351 v[0]=9.04 p=[0 0 1 0 3 0 1 0 0 0] 861 3298
alpha=0.20 iter=534 v[0]=22.87 p=[0 0 1 0 3 0 1 0 0 0] 771 1840
alpha=0.30 iter=664 v[0]=40.86 p=[0 0 1 0 3 0 1 0 0 0] 782 8466
alpha=0.35 iter=718 v[0]=52.17 p=[0 0 1 0 3 0 1 0 0 0] 773 8841
alpha=0.40 iter=763 v[0]=65.48 p=[0 0 1 0 3 0 1 0 0 0] 778 6225
alpha=0.45 iter=795 v[0]=80.41 p=[0 0 1 0 3 0 1 0 0 0] 733 7236
alpha=0.49 iter=804 v[0]=91.34 p=[0 0 1 0 3 0 1 0 0 0] 660 6247


In [5]:
def rows():
    for i, a in enumerate(alpha):
        rpp = mdp[i].reward_per_progress(vi[i]["vi_policy"], eps=0.0001)
        yield dict(alpha=a) | rpp


pandas.DataFrame(rows())

/home/patrik/devel/cpr/mdp/mdp.py:236: RuntimeWarning: invalid value encountered in divide
  next_rpp = sum(numpy.multiply(ss, reward[next,] / progress[next, ]))


,alpha,ss_time,rpp,rpp_iter,rpp_time
0,0.01,0.883917,0.010067,20,0.122633
1,0.05,0.674404,0.052653,25,0.134483
2,0.10,0.702096,0.110991,28,0.143254
3,0.20,0.622071,0.245424,22,0.128680
4,0.30,0.689885,0.397697,18,0.134456
5,0.35,0.643224,0.537087,92,0.332052
6,0.40,0.618296,0.603765,18,0.128394
7,0.45,0.573896,0.749803,22,0.130033
8,0.49,0.717902,0.878173,17,0.122229
